In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd


invoice       = pd.read_parquet("dataset_creados\\invoice.parquet")
invoice_line  = pd.read_parquet("dataset_creados\\invoiceLine.parquet")
partner       = pd.read_parquet("dataset_creados\\partner.parquet")
product       = pd.read_parquet("dataset_creados\\product.parquet")
payment       = pd.read_parquet("dataset_creados\\payment.parquet")
payment_app   = pd.read_parquet("dataset_creados\\paymentApplication.parquet")

invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 300 non-null    int64         
 1   partner_id         300 non-null    int64         
 2   currency_id        300 non-null    int64         
 3   invoice_type       300 non-null    object        
 4   invoice_date       300 non-null    datetime64[ns]
 5   due_date           300 non-null    datetime64[ns]
 6   total_amount       300 non-null    int64         
 7   invoice_id         146 non-null    float64       
 8   amount_paid        300 non-null    float64       
 9   remaining_balance  300 non-null    float64       
 10  payment_state      300 non-null    object        
dtypes: datetime64[ns](2), float64(3), int64(4), object(2)
memory usage: 25.9+ KB


In [3]:
hoy = pd.Timestamp.now()

filtro = invoice.loc[(invoice["invoice_type"] == "out")&
    (invoice["remaining_balance"] > 0)]
filtro["dias"] = (hoy-filtro["due_date"]).dt.days

#agrupamos por días 

etiquetas = ["0–30", "31–60", "61–90", ">90 días"]
lista = []

for i in filtro["dias"]:
    if  i >=0 and i < 30:
        lista.append(etiquetas[0])
    elif i >= 30 and i <= 60:
        lista.append(etiquetas[1])
    elif i > 60 and i <= 90:
        lista.append(etiquetas[2])
    else:
        lista.append(etiquetas[3])


filtro["butcket"] = lista
filtro


C:\Windows\Temp\ipykernel_17516\1551657628.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro["dias"] = (hoy-filtro["due_date"]).dt.days
C:\Windows\Temp\ipykernel_17516\1551657628.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro["butcket"] = lista


,id,partner_id,currency_id,invoice_type,invoice_date,due_date,total_amount,invoice_id,amount_paid,remaining_balance,payment_state,dias,butcket
0,1,33,2,out,2025-08-16 11:01:29.836924,2025-08-11 11:01:29.836924,2108,NaN,0.0,2108.0,unpaid,100,>90 días
2,3,12,1,out,2025-04-24 11:01:29.836924,2025-06-02 11:01:29.836924,14413,NaN,0.0,14413.0,unpaid,170,>90 días
4,5,36,2,out,2025-06-29 11:01:29.836924,2025-08-19 11:01:29.836924,18130,5.0,3815.0,14315.0,partial,92,>90 días
5,6,19,1,out,2024-09-12 11:01:29.836924,2024-10-03 11:01:29.836924,17245,6.0,184.0,17061.0,partial,412,>90 días
6,7,1,2,out,2024-05-01 11:01:29.836924,2024-05-26 11:01:29.836924,5928,7.0,3734.0,2194.0,partial,542,>90 días
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,289,37,2,out,2025-03-06 11:01:29.836924,2025-04-10 11:01:29.836924,12306,NaN,0.0,12306.0,unpaid,223,>90 días
291,292,26,1,out,2025-08-06 11:01:29.836924,2025-08-25 11:01:29.836924,28279,292.0,1642.0,26637.0,partial,86,61–90
292,293,4,1,out,2025-06-11 11:01:29.836924,2025-07-09 11:01:29.836924,8453,293.0,2808.0,5645.0,partial,133,>90 días
294,295,18,1,out,2024-02-09 11:01:29.836924,2024-03-16 11:01:29.836924,23458,NaN,0.0,23458.0,unpaid,613,>90 días


In [4]:
agrupado = filtro.groupby("butcket")["remaining_balance"].sum()
agrupado


vencidos_buckets = ["31–60", "61–90", ">90 días"]

saldo_vencido = agrupado.loc[vencidos_buckets].sum()
saldo_total = agrupado.sum()

porcentaje_vencido = (saldo_vencido / saldo_total) * 100
porcentaje_vencido


np.float64(92.28544066519085)

In [5]:
vencidos = filtro[filtro["dias"] > 0]

top10 = (
    vencidos.groupby("partner_id")["remaining_balance"].sum().sort_values(ascending=False).head(10)
)

top10

partner_id
33    128964.0
36     91252.0
15     75829.0
40     70053.0
19     61362.0
22     60884.0
34     55363.0
37     54416.0
4      52982.0
35     52583.0
Name: remaining_balance, dtype: float64

In [8]:
#CREAMOS UNA COLUMNA PARA PODER VALIR SI TANTO COMO STATUS COMO REMAIN BALANCE ETÁN CORRECTAMENTE INGRESADOS

errores_pago = filtro[
    (filtro["payment_state"] == "paid") &
    (filtro["remaining_balance"] > 0)
]

errores_pago

,id,partner_id,currency_id,invoice_type,invoice_date,due_date,total_amount,invoice_id,amount_paid,remaining_balance,payment_state,dias,butcket


ESTA ES LAS ECCION DE FLUJOS DE CAJA 

In [10]:



data_invoice = invoice.copy()


# Facturas OUT
facturas_out = data_invoice[
    (data_invoice["invoice_type"] == "out") &
    (data_invoice["remaining_balance"] > 0)
].copy()
hoy = pd.Timestamp.today()
semanas = pd.date_range(hoy, periods=9, freq="W")




facturas_out["semana"] = "*"


for i in range(1, 9):
    inicio = semanas[i-1]
    fin = semanas[i]
    
    facturas_out.loc[
        (facturas_out["due_date"] >= inicio) &
        (facturas_out["due_date"] < fin),
        "semana"
    ] = f"Semana {i}"
entradas = (
    facturas_out.groupby("semana")["remaining_balance"].sum().reset_index().rename(columns={"remaining_balance": "entrada"})
)

entradas

,semana,entrada
0,*,1623497.0
1,Semana 1,22370.0
2,Semana 2,31449.0
3,Semana 3,19292.0
4,Semana 5,23098.0
5,Semana 6,6355.0


In [11]:
facturas_in = data_invoice[
    (data_invoice["invoice_type"] == "in") &
    (data_invoice["remaining_balance"] > 0)
].copy()

facturas_in["semana"] = "*"

for i in range(1, 9):
    inicio = semanas[i-1]
    fin = semanas[i]
    
    facturas_in.loc[
        (facturas_in["due_date"] >= inicio) &
        (facturas_in["due_date"] < fin),
        "semana"
    ] = f"Semana {i}"
salidas = (
    facturas_in.groupby("semana")["remaining_balance"]
    .sum()
    .reset_index()
    .rename(columns={"remaining_balance": "salida"})
)
salidas


,semana,salida
0,*,1595280.0
1,Semana 1,6214.0
2,Semana 2,26774.0
3,Semana 3,41526.0
4,Semana 5,21429.0


In [12]:
flujo = entradas.merge(salidas, on="semana", how="outer")
flujo = flujo.fillna(0) 

flujo["neto"] = flujo["entrada"] - flujo["salida"]

flujo


hoy = pd.Timestamp.today()
limite_30 = hoy + pd.Timedelta(days=30)

cxc_corriente = data_invoice[
    (data_invoice["invoice_type"] == "out") &
    (data_invoice["remaining_balance"] > 0) &
    (data_invoice["due_date"] >= hoy) &
    (data_invoice["due_date"] <= limite_30)
]["remaining_balance"].sum()

cxc_corriente
entradas_30 = facturas_out[
    (facturas_out["due_date"] >= hoy) &
    (facturas_out["due_date"] <= limite_30)
]["remaining_balance"].sum()


cxc_corriente, entradas_30

(np.float64(86117.0), np.float64(86117.0))

AHORA CALCULAMOS LAS GANANCIAS


In [13]:
data_invoice = pd.read_parquet("dataset_creados/Invoice.parquet")
data_invoice_line = pd.read_parquet("dataset_creados/InvoiceLine.parquet")
data_product = pd.read_parquet("dataset_creados/Product.parquet")
data_partner = pd.read_parquet("dataset_creados/Partner.parquet")

In [14]:
data_invoice

,id,partner_id,currency_id,invoice_type,invoice_date,due_date,total_amount,invoice_id,amount_paid,remaining_balance,payment_state
0,1,33,2,out,2025-08-16 11:01:29.836924,2025-08-11 11:01:29.836924,2108,NaN,0.0,2108.0,unpaid
1,2,36,1,in,2025-01-21 11:01:29.836924,2025-03-20 11:01:29.836924,27796,2.0,2331.0,25465.0,partial
2,3,12,1,out,2025-04-24 11:01:29.836924,2025-06-02 11:01:29.836924,14413,NaN,0.0,14413.0,unpaid
3,4,14,1,in,2025-05-02 11:01:29.836924,2025-06-26 11:01:29.836924,2542,4.0,607.0,1935.0,partial
4,5,36,2,out,2025-06-29 11:01:29.836924,2025-08-19 11:01:29.836924,18130,5.0,3815.0,14315.0,partial
...,...,...,...,...,...,...,...,...,...,...,...
295,296,8,1,out,2025-09-16 11:01:29.836924,2025-11-10 11:01:29.836924,3516,296.0,690.0,2826.0,partial
296,297,21,2,in,2024-04-16 11:01:29.836924,2024-06-03 11:01:29.836924,4724,297.0,1108.0,3616.0,partial
297,298,28,2,in,2024-12-28 11:01:29.836924,2025-01-31 11:01:29.836924,14064,298.0,2666.0,11398.0,partial
298,299,31,1,in,2024-02-01 11:01:29.836924,2024-03-17 11:01:29.836924,24723,NaN,0.0,24723.0,unpaid


In [15]:
df_producto_invoice = data_invoice_line.merge(
    data_product,
    left_on="product_id",
    right_on="id",
    how="left"
)

df_producto_invoice["margen_unitario"] = df_producto_invoice["price"] - df_producto_invoice["cost_per_unit"]
df_producto_invoice["margen_total"] = df_producto_invoice["margen_unitario"] * df_producto_invoice["quantity"]

df = df_producto_invoice.merge(
    data_invoice[["id", "partner_id", "invoice_type"]],
    left_on="invoice_id",
    right_on="id",
    how="left",
    suffixes=("", "_inv")
) 
df = df[df["invoice_type"] == "out"]

In [16]:
margen_producto = (
    df.groupby("product_id")[["margen_unitario", "margen_total"]]
    .sum()
    .reset_index()
)

margen_producto = margen_producto.merge(
    data_product[["id", "product_name"]],
    left_on="product_id",
    right_on="id",
    how="left"
)

margen_producto

,product_id,margen_unitario,margen_total,id,product_name
0,1,53280,293040,1,Producto_1
1,2,22016,123152,2,Producto_2
2,3,8760,38325,3,Producto_3
3,4,25236,124778,4,Producto_4
4,5,-16856,-80668,5,Producto_5
5,6,16425,84096,6,Producto_6
6,7,13860,74745,7,Producto_7
7,8,27972,136863,8,Producto_8
8,9,33480,178560,9,Producto_9
9,10,28782,133947,10,Producto_10


In [17]:
margen_cliente = (
    df.groupby("partner_id")["margen_total"]
    .sum()
    .reset_index()
)


margen_cliente = margen_cliente.merge(
    data_partner[["id", "display_name"]],
    left_on="partner_id",
    right_on="id",
    how="left"
)

margen_cliente


,partner_id,margen_total,id,display_name
0,1,25968,1,Cliente_1
1,2,12676,2,Cliente_2
2,3,2064,3,Cliente_3
3,4,23912,4,Cliente_4
4,5,54600,5,Cliente_5
5,6,29225,6,Cliente_6
6,7,50108,7,Cliente_7
7,8,24240,8,Cliente_8
8,9,38069,9,Cliente_9
9,10,20177,10,Cliente_10


In [18]:
productos_perdida = margen_producto[margen_producto["margen_total"] < 0]
productos_perdida

,product_id,margen_unitario,margen_total,id,product_name
4,5,-16856,-80668,5,Producto_5


In [19]:
clientes_perdida = margen_cliente[margen_cliente["margen_total"] < 0]
clientes_perdida

,partner_id,margen_total,id,display_name


DSO / DPO / DIO / CCC

In [33]:

invoice = pd.read_parquet("dataset_creados/Invoice.parquet")
invoice_line = pd.read_parquet("dataset_creados/InvoiceLine.parquet")
partner = pd.read_parquet("dataset_creados/Partner.parquet")
product = pd.read_parquet("dataset_creados/Product.parquet")
payment = pd.read_parquet("dataset_creados/Payment.parquet")
payment_app = pd.read_parquet("dataset_creados/PaymentApplication.parquet")
exchange = pd.read_parquet("dataset_creados/ExchangeRate.parquet")
journal = pd.read_parquet("dataset_creados/JournalLine.parquet")


In [20]:



facturas_out = invoice[invoice["invoice_type"] == "out"].copy()
facturas_in = invoice[invoice["invoice_type"] == "in"].copy()


# 2. UNIR PAGO + PAYMENT APP

pagos_aplicados = payment_app.merge(
    payment,
    left_on="payment_id",
    right_on="id",
    how="left",
    suffixes=("_app", "_pay")
)


# 3. DSO 

pagos_ventas = pagos_aplicados.merge(
    facturas_out[["id", "invoice_date"]],
    left_on="invoice_id",
    right_on="id",
    how="inner"
)

pagos_ventas["dias_cobro"] = (
    pagos_ventas["payment_date"] - pagos_ventas["invoice_date"]
).dt.days

DSO = pagos_ventas["dias_cobro"].mean()


# 4. DPO )

pagos_compras = pagos_aplicados.merge(
    facturas_in[["id", "invoice_date"]],
    left_on="invoice_id",
    right_on="id",
    how="inner"
)

pagos_compras["dias_pago"] = (
    pagos_compras["payment_date"] - pagos_compras["invoice_date"]
).dt.days

DPO = pagos_compras["dias_pago"].mean()


# 5. DIO 

DIO = 0

# 6. CCC (Cash Conversion Cycle)

CCC = DIO + DSO - DPO

#SERIE MENSUALIZADA (DSO y DPO)

dso_mensual = pagos_ventas.groupby(
    pagos_ventas["payment_date"].dt.to_period("M")
)["dias_cobro"].mean().reset_index()

dso_mensual = dso_mensual.rename(columns={"dias_cobro": "DSO"})

dpo_mensual = pagos_compras.groupby(
    pagos_compras["payment_date"].dt.to_period("M")
)["dias_pago"].mean().reset_index()

dpo_mensual = dpo_mensual.rename(columns={"dias_pago": "DPO"})


# 8. ALERTA SI DSO > 90 DÍAS

alerta_dso = dso_mensual[dso_mensual["DSO"] > 90]

print("DSO =", DSO)
print("DPO =", DPO)
print("DIO =", DIO)
print("CCC =", CCC)

print("\n--- Serie mensual DSO ---")
display(dso_mensual)

print("\n--- Serie mensual DPO ---")
display(dpo_mensual)

if len(alerta_dso) > 0:
    print(" ALERTA: DSO supera los 90 días en estos meses:")
    display(alerta_dso)
else:
    print("\n DSO  No supera los 90 días en ningún mes.")


DSO = -38.91919191919192
DPO = -24.415841584158414
DIO = 0
CCC = -14.503350335033502

--- Serie mensual DSO ---


,payment_date,DSO
0,2023-12,-275.800000
1,2024-01,-312.250000
2,2024-02,-514.800000
3,2024-03,-299.285714
4,2024-04,-110.750000
5,2024-05,-257.666667
6,2024-06,-81.000000
7,2024-07,-277.666667
8,2024-08,-301.250000
9,2024-09,-191.000000



--- Serie mensual DPO ---


,payment_date,DPO
0,2023-11,-405.666667
1,2023-12,-213.000000
2,2024-01,-265.857143
3,2024-02,-294.000000
4,2024-03,-191.000000
5,2024-04,-272.500000
6,2024-05,-109.750000
7,2024-06,-199.000000
8,2024-07,-233.000000
9,2024-08,-358.500000


 ALERTA: DSO supera los 90 días en estos meses:


,payment_date,DSO
14,2025-02,308.00
17,2025-05,143.00
18,2025-06,282.60
19,2025-07,345.50
20,2025-08,438.50
21,2025-09,306.25
22,2025-10,479.25
23,2025-11,371.00


5. Panel ejecutivo mensual

In [ ]:

# 2. Preparar columnas de periodo (año-mes)
invoice["year_month"] = invoice["invoice_date"].dt.to_period("M")


# 3. Ventas mensuales (solo OUT)

ventas = invoice[invoice["invoice_type"] == "out"].copy()
ventas_mensuales = (
    ventas.groupby("year_month")["total_amount"]
    .sum()
    .reset_index()
    .rename(columns={"total_amount": "ventas"})
)


# 4. Margen mensual (OUT) usando InvoiceLine + Product + Invoice

df = invoice_line.merge(
    product,
    left_on="product_id",
    right_on="id",
    how="left"
)

df["margen_unitario"] = df["price"] - df["cost_per_unit"]
df["margen_total"]    = df["margen_unitario"] * df["quantity"]

df = df.merge(
    invoice[["id", "invoice_date", "invoice_type", "year_month"]],
    left_on="invoice_id",
    right_on="id",
    how="left",
    suffixes=("", "_inv")
)

df_out = df[df["invoice_type"] == "out"].copy()

margen_mensual = (
    df_out.groupby("year_month")["margen_total"]
    .sum()
    .reset_index()
    .rename(columns={"margen_total": "margen"})
)


# 5. AR (cuentas por cobrar) vigentes por mes
#    Facturas OUT con saldo pendiente

ar = invoice[
    (invoice["invoice_type"] == "out") &
    (invoice["remaining_balance"] > 0)
].copy()

ar_mensual = (
    ar.groupby("year_month")["remaining_balance"]
    .sum()
    .reset_index()
    .rename(columns={"remaining_balance": "AR"})
)


# 6. AP (cuentas por pagar) vigentes por mes
#    Facturas IN con saldo pendiente

ap = invoice[
    (invoice["invoice_type"] == "in") &
    (invoice["remaining_balance"] > 0)
].copy()

ap_mensual = (
    ap.groupby("year_month")["remaining_balance"]
    .sum()
    .reset_index()
    .rename(columns={"remaining_balance": "AP"})
)


# 7. Pagos aplicados: base para flujo y DSO/DPO

pagos = payment_app.merge(
    invoice[["id", "invoice_type", "invoice_date"]],
    left_on="invoice_id",
    right_on="id",
    how="left"
).merge(
    payment[["id", "payment_date"]],
    left_on="payment_id",
    right_on="id",
    how="left",
    suffixes=("_inv", "_pay")
)

pagos["year_month"] = pagos["payment_date"].dt.to_period("M")
pagos["dias"] = (pagos["payment_date"] - pagos["invoice_date"]).dt.days


#  Flujo neto 
#    Entradas: pagos aplicados a OUT( es lo de la empresa)
#    Salidas:  pagos aplicados a IN ( es lo que pagas a probablemente proveedores)

entradas_mensuales = (
    pagos[pagos["invoice_type"] == "out"]
    .groupby("year_month")["allocated_amount"]
    .sum()
    .reset_index()
    .rename(columns={"allocated_amount": "entradas"})
)

salidas_mensuales = (
    pagos[pagos["invoice_type"] == "in"]
    .groupby("year_month")["allocated_amount"]
    .sum()
    .reset_index()
    .rename(columns={"allocated_amount": "salidas"})
)

flujo_mensual = entradas_mensuales.merge(
    salidas_mensuales,
    on="year_month",
    how="outer"
).fillna(0)

flujo_mensual["flujo_neto"] = flujo_mensual["entradas"] - flujo_mensual["salidas"]

# 9. DSO y DPO mensuales

dso_mensual = (
    pagos[pagos["invoice_type"] == "out"]
    .groupby("year_month")["dias"]
    .mean()
    .reset_index()
    .rename(columns={"dias": "DSO"})
)

dpo_mensual = (
    pagos[pagos["invoice_type"] == "in"]
    .groupby("year_month")["dias"]
    .mean()
    .reset_index()
    .rename(columns={"dias": "DPO"})
)


# 10. Armar el panel ejecutivo mensual

panel_mensual = (
    ventas_mensuales
    .merge(margen_mensual, on="year_month", how="outer")
    .merge(ar_mensual,      on="year_month", how="outer")
    .merge(ap_mensual,      on="year_month", how="outer")
    .merge(flujo_mensual,   on="year_month", how="outer")
    .merge(dso_mensual,     on="year_month", how="outer")
    .merge(dpo_mensual,     on="year_month", how="outer")
)

panel_mensual = panel_mensual.fillna(0)

# Pasar year_month a string para visualizar más bonito
panel_mensual["year_month"] = panel_mensual["year_month"].astype(str)

panel_mensual = panel_mensual.sort_values("year_month").reset_index(drop=True)
panel_mensual


,year_month,ventas,margen,AR,AP,entradas,salidas,flujo_neto,DSO,DPO
0,2023-11,32448,10998,30410.0,24650.0,3808,1531.0,2277.0,-150.000000,-358.000000
1,2023-12,66236,20014,62491.0,67861.0,4102,1320.0,2782.0,-365.000000,-331.200000
2,2024-01,42849,5850,42534.0,56398.0,7122,4588.0,2534.0,-219.800000,-356.285714
3,2024-02,31630,2489,23746.0,33143.0,1275,6142.0,-4867.0,-523.000000,-252.000000
4,2024-03,17652,4738,12437.0,50689.0,6425,10607.0,-4182.0,-176.000000,-152.857143
5,2024-04,47179,15819,32932.0,82588.0,4909,3442.0,1467.0,-127.400000,-360.000000
6,2024-05,43253,-1638,39698.0,38792.0,579,9529.0,-8950.0,-351.000000,-136.166667
7,2024-06,25753,2774,20532.0,49856.0,6250,1578.0,4672.0,36.750000,-305.000000
8,2024-07,41630,6831,35095.0,26251.0,1341,9196.0,-7855.0,95.000000,-91.714286
9,2024-08,19939,-2119,13465.0,51166.0,1807,8384.0,-6577.0,-150.333333,-80.000000


In [21]:
!pip freeze > requirements.txt